# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Задание будет выполняться в последовательности определенной заголовками

## Откройте и изучите файл

In [18]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

Импорт библиотек

In [19]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В выборке порядка трех тысяч записей. По условиям задачи предобработка данных не требуется

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

In [21]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Выделены признаки и целевой признак

In [22]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4, random_state=12345)
features_train.shape

(1928, 4)

Создана обучающая выборка в рекомендованном размере - 60% общей

In [23]:
features_valid.shape

(1286, 4)

In [24]:
features_test, features_valid, target_test, target_valid = train_test_split(features_valid, target_valid, test_size=0.5, random_state=12345)
features_valid.shape

(643, 4)

Создана валидационная выборка в рекомендованном размере - 20% общей

In [25]:
features_test.shape

(643, 4)

Создана тестовая выборка в рекомендованном размере - 20% общей

## Исследуйте модели

На данном этапе будут последовательно применены три модели обучения: дерево решений, случайный лес и логистическая регрессия с перебором параметров там где это применимо. Полученные результаты будут опробованы на валидационной выборке

In [26]:
tree_model = None
tree_result = 0
for depth in range(1,101):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > tree_result:
        tree_model = model
        tree_result = result
        tree_depth = depth
print('Accuracy лучшей модели:', tree_result )  
print('Оптимальная глубина дерева:', tree_depth)

Accuracy лучшей модели: 0.7993779160186625
Оптимальная глубина дерева: 7


При использовании дерева решений на валидационной выборке наилучшая accuracy в 80% достигается при глубине дерева в 7 узлов. Следующая модель - случайный лес

In [36]:
forest_model = None
forest_result = 0
for est in range(1,51):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > forest_result:
        forest_model = model
        forest_result = result 
        forest_est = est
print('Accuracy лучшей модели:', forest_result )  
print('Оптимальное количество деревьев:', forest_est)    

Accuracy лучшей модели: 0.7962674961119751
Оптимальное количество деревьев: 48


Для случайного леса достижимая accuracy также составляет примерно 80% и достигается при 48 деревьях. При этом близкое по качеству решение с accuracy в 78% может быть достигнуто уже при 8 деревьях. Остается логистическая регрессия

In [28]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=500000)
model.fit(features_train, target_train)
log_result = model.score(features_valid, target_valid)
print('Accuracy модели:', log_result) 

Accuracy модели: 0.6842923794712286


Логистическая регрессия дает минимальную точность на валидационной выборке - 68%. Но она достигается уже при 50 итерациях и далее не растет. На валидационной выборке наилучший результат показало дерево решений, с минимальным отрывом от нее идет случайный лес и далее с большим отставанием логистическая регрессия. На следующем шаге лучшая модель - дерево решений будет проверено на тестовой выборке с гиперпараметрами полученными на валидационной выборке

## Проверьте модель на тестовой выборке

In [29]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7)
model.fit(features_train, target_train)
tree_test = model.score(features_test, target_test)
print('Accuracy модели:', tree_test) 

Accuracy модели: 0.7822706065318819


Наилучшая модель - дерево решений с 7 деревьями на тестовой выборке показала accuracy в 78% Целевой показатель в 75% достигнут и превышен, соответственно, модель дерева решений с 7 деревьями отвечает поставленному в условии задачи требованию